In [1]:
import torch
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader

In [2]:
from custom.models import seg_net
from custom.utils import rle_encoding
from custom.datasets import nuclei_dataset, nuclei_data

/root/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.16) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
import pandas as pd
import numpy as np

In [4]:
dat = nuclei_data(path='../data/stage1_test/', test=True)
test_loader = DataLoader(nuclei_dataset(dat, test=True), batch_size=1)

In [5]:
model = seg_net()
model.load_state_dict(torch.load('./models/segnet_2'))
model.cuda()

seg_net(
  (conv_block_1): conv357_block(
    (conv3): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv5): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (conv7): Conv2d(1, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (norm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
  )
  (conv_block_2): conv357_block(
    (conv3): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv5): Conv2d(96, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (conv7): Conv2d(96, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (norm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
  )
  (conv_block_3): conv357_block(
    (conv3): Conv2d(96, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv5): Conv2d(96, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (conv7): Conv2d(96, 1, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (norm): BatchNorm2d(3, eps=1e-05, momentum=

In [6]:
to_cuda_var = lambda x: Variable(x.float()).cuda()
from torch.nn.functional import threshold
from torchvision.utils import save_image
def test():
    model.eval()
    rle_codes = []
    for i, im in enumerate(test_loader):
        input = to_cuda_var(im)
        output = model(input)
        label = threshold(-threshold(output, .5, 0, inplace=True), -.5, 1, inplace=True)
        save_image(label.data.cpu(),'./results_test/' + str(i) + '.png', nrow=1)
        
        label = label.data.cpu().numpy()
        rle_codes.append(rle_encoding(label))

    return rle_codes

In [7]:
rle_codes = test()
im_ids = dat.im_ids

In [8]:
df = pd.DataFrame({'ImageId': im_ids, 'EncodedPixels': rle_codes})
df = df[['ImageId', 'EncodedPixels']]

In [9]:
df.to_csv('submission.csv', index=False)

In [10]:
# df = pd.DataFrame.from_csv('submission.csv')